# Momentum_Strategy

In [ ]:
# 设定data路径
import os

os.chdir(r'F:\Study\Study Files\05大三上\金融统计分析\实验课1\1_Rawdata')

current_path = os.getcwd()
print(current_path)

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('TRD_Mnth.csv')
#data = data.drop(['Markettype'], axis=1)

## 数据清洗

In [4]:
# data_raw.head()

# des = data_raw.describe(include='all')
# print(des)

In [5]:
# data_raw = data_raw.dropna()

# des = data_raw.describe(include='all')
# print(des)

### 剔除对应股票代码：没有完整回测期以及前后半年（回测期为`1992/1/1-2021/12/1`）

In [6]:
# start_date = '1991-06'
# end_date = '2022-06'

# # 将Trdmnt列转换为datetime类型
# data_raw['Trdmnt'] = pd.to_datetime(data_raw['Trdmnt'], format='%Y-%m')

# '''
# # 筛选出在指定时间范围内的数据
# data_filtered1 = data_raw[(data_raw['Trdmnt'] >= start_date) & (data_raw['Trdmnt'] <= end_date)]
# '''
# data_filtered1 = data_raw

# # 按Stkcd分组并计算每组的时间跨度
# grouped = data_filtered1.groupby('Stkcd')['Trdmnt'].agg(['min', 'max'])

# # 筛选出符合条件的股票代码
# valid_stkcd = grouped[(grouped['min'] <= start_date) & (grouped['max'] >= end_date)].index

# # 保留符合条件的股票代码的数据
# data_result = data_filtered1[data_filtered1['Stkcd'].isin(valid_stkcd)]

# # data_result现在包含了符合条件的股票代码的数据
# data_result.head()

### 检查`Stkcd`对应月份是否连续
- 结果：没有任何股票代码对应的月份是完全连续的

In [7]:
# # 创建data_result的副本
# data_result_copy = data_result.copy()

# # 提取Trdmnt中的月份
# data_result_copy['month'] = data_result_copy['Trdmnt'].dt.month

# # 定义一个函数，检查月份是否连续
# def is_month_consecutive(group):
#     sorted_months = group.sort_values()
#     diff = np.diff(sorted_months)
#     return np.all(diff == 1) or np.all(diff == -11)

# # 按照Stkcd进行分组，在组内检查month是否连续
# consecutive_check = data_result_copy.groupby('Stkcd')['month'].apply(is_month_consecutive)

# print(consecutive_check)

# # 筛选出月份连续的股票代码
# consecutive_stkcd = consecutive_check[consecutive_check].index
# print(consecutive_stkcd)

# # 保留月份连续的股票代码的数据
# data_1 = data_result_copy[data_result_copy['Stkcd'].isin(consecutive_stkcd)]

# # data_1现在包含了符合条件的且月份连续的股票代码的数据
# data_1.head()

- 填充数据

In [8]:
# import pandas as pd
# import numpy as np

# # 创建data_result的副本
# data_result_copy = data_result.copy()

# def fill_missing_months(group):
#     group = group.set_index('Trdmnt')
#     group = group.resample('M').asfreq()
#     group['Stkcd'] = group['Stkcd'].fillna(method='ffill').fillna(method='bfill')
#     group['Msmvosd'] = group['Msmvosd'].fillna(method='ffill').fillna(method='bfill')
#     group['Mretwd'] = group['Mretwd'].fillna(0)
#     return group


# # 按照Stkcd进行分组，补充缺失月份
# filled_data = data_result_copy.groupby('Stkcd', group_keys=False).apply(fill_missing_months).reset_index()

# # filled_data现在包含了补充缺失月份的数据
# print(filled_data)


## 策略构建

### BingAI

In [9]:
# # 导入所需的库
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # 定义形成期和持有期的列表
# formation_periods = [1, 3, 6]
# holding_periods = [1, 3, 6]

# # 读取TRD_Mnth.csv文件，将'Trdmnt'列转换为日期格式，并设置为索引
# data = pd.read_csv("TRD_Mnth.csv", parse_dates=["Trdmnt"], index_col="Trdmnt")

# # 计算每只股票在每个形成期内的累积收益率
# returns = {}
# for fp in formation_periods:
#     # 对于每个形成期，先将数据按照股票代码和交易日期进行分组和排序
#     grouped = data.groupby(["Stkcd", "Trdmnt"]).first().sort_index()
#     # 然后，使用shift函数将数据向后移动fp个月，以便计算形成期内的累积收益率
#     shifted = grouped.shift(fp)
#     # 接着，使用pct_change函数计算形成期内的累积收益率，并将结果存储到returns字典中
#     returns[fp] = grouped["Mretwd"].pct_change().div(shifted["Mretwd"])

# # 创建一个空的DataFrame来存储不同形成期和持有期下的多空组合收益率
# results = pd.DataFrame(index=formation_periods, columns=holding_periods)

# # 对于每个形成期和持有期的组合，执行以下步骤：
# for fp in formation_periods:
#     for hp in holding_periods:
#         # 根据累积收益率对股票进行分组和排序
#         groups = returns[fp].groupby(level=0).rank(ascending=False)
#         # 构建多空组合，并计算其在每个持有期内的收益率
#         portfolio = data["Mretwd"].groupby(level=0).shift(-hp)
#         portfolio = portfolio * (groups <= 50) - portfolio * (groups > 450)
#         portfolio = portfolio.groupby(level=0).mean()
#         # 计算多空组合的年化收益率，并存储到结果DataFrame中
#         annualized_return = (portfolio + 1).prod() ** (12 / (fp * len(portfolio))) - 1
#         results.loc[fp, hp] = annualized_return

# # 绘制结果DataFrame的表格
# plt.figure(figsize=(10, 6))
# plt.table(cellText=results.round(4).values, rowLabels=results.index, colLabels=results.columns, loc="center")
# plt.axis("off")
# plt.title("Momentum Strategy Annualized Returns")
# plt.show()


### GPT4 by 20 times use perday

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 读取数据
data = pd.read_csv('TRD_Mnth.csv')

# 确保数据按股票代码和日期排序
data = data.sort_values(['Stkcd', 'Trdmnt'])
data['Trdmnt'] = pd.to_datetime(data['Trdmnt'])

# 设置回测期
data = data[(data['Trdmnt'] >= '1992-01-01') & (data['Trdmnt'] <= '2021-12-31')]

# 初始化一个3x3的DataFrame来保存不同策略的收益
result = pd.DataFrame(index=[1,3,6], columns=[1,3,6])

# 开始循环：不同形成期和持有期
for form_period in [1,3,6]:
    for hold_period in [1,3,6]:
        
        all_returns = []
        
        # 遍历每一个可以进行交易的日期
        for end_date in pd.date_range(start='1992-01-01', end='2021-12-31', freq='M'):
            start_date = end_date - pd.DateOffset(months=form_period)  # 形成期起始日期
            hold_end_date = end_date + pd.DateOffset(months=hold_period)  # 持有期结束日期
            
            if hold_end_date > data['Trdmnt'].max():
                continue
            
            # 计算形成期内的累计收益
            mask = (data['Trdmnt'] >= start_date) & (data['Trdmnt'] < end_date)
            form_returns = data.loc[mask].groupby('Stkcd')['Mretwd'].apply(lambda x: (1 + x).prod() - 1)
            
            winners = form_returns.nlargest(10).index
            losers = form_returns.nsmallest(10).index
            
            # 计算持有期收益
            mask = (data['Trdmnt'] > end_date) & (data['Trdmnt'] <= hold_end_date)
            hold_returns = data.loc[mask].groupby(['Stkcd', 'Trdmnt'])['Mretwd'].apply(lambda x: ((1 + x).prod())**(1/hold_period) - 1).unstack().mean(axis=1)
            
            winner_returns = hold_returns.loc[winners].mean()
            loser_returns = hold_returns.loc[losers].mean()
            
            # 计算策略收益：赢家收益 - 输家收益
            strategy_return = winner_returns - loser_returns
            
            all_returns.append(strategy_return)
            
        result.loc[form_period, hold_period] = np.mean(all_returns)

# 展示结果
sns.heatmap(result, annot=True, fmt=".2%", cmap="RdYlGn", center=0)
plt.title("Momentum Strategy Returns (Jegadeesh and Titman, 1993)")
plt.xlabel("Holding Period (months)")
plt.ylabel("Formation Period (months)")
plt.show()